# End-to-end handwriting-to-LaTeX Demo

## Summary

We are demoing the process of taking handwritten math and converting to LaTeX using the top tools found in the research phase. The writing and input will be done using tkinter, where writing will occur in a separate window. The math OCR will be done with MathPix's OCR API.

The first test of the full workflow produced a rendered equation from a handwritten input whith about 9/11 characacters correctly. I recognize two problems contributing to this inaccuracy. The first, is the canvas drawing is not the exact image being sent. A background PIL canvas is silently keeping track of the drawing using line objects native to PIL. These objects are not presented as smoothly as the tkinter counterparts see image 2 and image 3. Another source of error is inaccurate user input, ie, bad handwriting.  

Accuracy and hand-writing style aside, it does show we can enact an end-to-end solution for the product.

## Experiment start

In [1]:
# image manipulation
from tkinter import Tk, Canvas, ttk, Button
from tkinter import constants as con
from PIL import ImageGrab, ImageTk, ImageDraw
import PIL

# math
import string
import random
from IPython.display import Markdown as md

# system
import sys
import base64

# requests
import requests
import json

**Constants for requests**

In [2]:
app_id = #ask owner 
app_key = #ask owner

In [4]:
def rnd_image_filename(N=7):
    """
    Creates random strings to follow a file name for uniqueness. We allow 7 numeric characters 
    
    7^36 = 2.6515x10^30, which is how many letters and numbers in the english 
    alphabet exsit. A choice of 7 means there are this many unique permutations.
    It will nearly guarantee no two files will be named the same even on 
    back-to-back runs. 
    
    N: Integer.
    How many digits to append to a file name to make it unique.
    
    filename: String.
    Name of the file to be used 
    """
    
    filetag=''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
    filename="figures/canvas_img_"+filetag+".png"
    return filename

In [5]:
# unit test
for i in range(5):
    filename_test = rnd_image_filename()
    print(filename_test)

figures/canvas_img_59E1UJR.png
figures/canvas_img_6OJJSCZ.png
figures/canvas_img_BT21910.png
figures/canvas_img_4RAW2S4.png
figures/canvas_img_QC3B9VS.png


## Phase 1: tkinter implementation

This code weaves tkinter's tutorial showcase of the code with a few lines from a code from this video (https://www.youtube.com/watch?v=OdDCsxfI8S0).
The code block below 'Pilot Canvas code' combines what happens in the code blocks below 'Simple Canvas code' and 'Canvas w Save Button'.

## Simple Canvas code

- no buttons or any save feature, just drawing
- may remove # @ diag for the original version, I don't think they're necess.
- **< button -1 >** =  Button 1 is the leftmost button, button 2 is the middle button (where available), and button 3 the rightmost button.
- **< B1-Motion >** =  The mouse is moved, with mouse button 1 being held down (use B2 for the middle button, B3 for the right button).

In [ ]:
# CANVAS 1
def savePosn(event):
    global lastx, lasty
    lastx, lasty = event.x, event.y

def addLine(event):
    canvas.create_line((lastx, lasty, event.x, event.y))
    savePosn(event)

root = Tk()

# diag: don't actually need this.
# root.columnconfigure(0, weight=1)
# root.rowconfigure(0, weight=1)

canvas = Canvas(root, bg="white")
# diag: if you add this line, you don't need the grid
canvas.pack()

# see comment @ diag above
# canvas.grid(column=0, row=0, sticky=(con.N, con.W, con.E ,con.S))
canvas.bind("<Button-1>", savePosn)
canvas.bind("<B1-Motion>", addLine)

root.mainloop(0)

## Canvas w Save Button

- has the complete code from the youtube channel which will draw and save using a button.
- the draw feature is crap, which is why we are trying to infuse the other code into it
- modified to use **rnd_image_filename**

In [ ]:
# this is the pilot code

width = 600
height = 400
center = height//2
white = (255, 255, 255)
background = 'white'
green = (0,128,0)

def save():
    filename = rnd_image_filename()
    canvas_image.save(filename)

def paint(event):
    # python_green = "#476042"
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    cv.create_oval(x1, y1, x2, y2,fill="black",width=2)
    draw.line([x1, y1, x2, y2],fill="black",width=2)

root = Tk()

# Tkinter create a canvas to draw on
cv = Canvas(root, width=width, height=height, bg=background)
cv.pack()

# PIL create an empty image and draw object to draw on
# memory only, not visible
canvas_image = PIL.Image.new("RGB", (width, height), white)
draw = ImageDraw.Draw(canvas_image)

# do the Tkinter canvas drawings (visible)
# cv.create_line([0, center, width, center], fill='green')

cv.pack(expand=True, fill="both")
cv.bind("<B1-Motion>", paint)

# do the PIL image/draw (in memory) drawings
# draw.line([0, center, width, center], green)

# PIL image can be saved as .png .jpg .gif or .bmp file (among others)
# filename = "my_drawing.png"
# image1.save(filename)
button=Button(text="save",command=save)
button.pack()
root.mainloop()

print("Complete canvas input and image save.")


## Pilot Canvas Code

Combine elements from prior code to create a canvas w a save button with smooth line drawing.

In [17]:
def pilot_canvas(width = 800, height = 600, linecolor="BLACK"):
    
    linewidth = 3
    offset = (linewidth)/2
    white = (255,255,255)
    filename = {}

    def save(N = 5):
        filename["name"] = rnd_image_filename(N=N)
        canvas_image.save(filename["name"])
        print("File was saved as: ", filename["name"] )

    def savePosn(event):
        global lastx, lasty
        lastx, lasty = event.x, event.y

    def addLine(event):
        # the canvas call is what you see on screen
        canvas.create_line((lastx, lasty, event.x, event.y),
                            smooth = True, width = linewidth, fill = linecolor)
        # the draw call is in the background (invisible) capturing what will actually get converted.
        draw.line([lastx, lasty, event.x, event.y], fill = linecolor, width = linewidth, joint = 'curve')
        savePosn(event)

    root = Tk()

    # Tkinter create a canvas to draw on
    canvas = Canvas(root, bg="white", width=width, height=height)
    canvas.pack()

    # PIL create an empty image and draw object to draw on
    # memory only, not visible
    canvas_image = PIL.Image.new("RGB", (width, height), white)
    draw = ImageDraw.Draw(canvas_image)

    canvas.pack(expand = True, fill = "both")
    canvas.bind("<Button-1>", savePosn)
    canvas.bind("<B1-Motion>", addLine)

    # Add a save button
    button = Button(text = "Save Image",command = lambda: save(N = 7))
    button.pack()

    # Add an exit button
    # later....

    root.mainloop()

    return filename

## Phase 2: MathPix OCR API implementation

We implement code from 'top_ocr_tools_mathpix_snip' nb for submitting an API request to MathPix. 

In [15]:
def ocr_request(filename):
    dict_request={
            "src": "data:image/png",
            "formats": ["text", "data", "html"],
            "data_options": {
            "include_asciimath": True,
            "include_latex": True
            }
        }

    # put desired filename from earlier.
    file_path = filename["name"]
    image_uri = "data:image/png;base64," + base64.b64encode(open(file_path, "rb").read()).decode()

    # send a request
    r = requests.post("https://api.mathpix.com/v3/text",
        data=json.dumps({'src': image_uri}),
        headers={"app_id": app_id, 
                 "app_key": app_key,
                 "Content-type": "application/json"})

    print(json.dumps(json.loads(r.text), indent=4, sort_keys=True))
    
    json_return = json.loads(r.text)
    latex_return = json_return.get("latex_styled")
    
    print(latex_return)
    print()
    return latex_return

# Begin Testing

## Test Case 0:

Make up a random equation and try it out.
Return from random equation drawn on the canvas. The input call is omitted.


In [49]:
# returned: {
#   "text": "\\( \\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right. \\)",
#   "confidence": 0.47379128643166557,
#   "is_printed": false,
#   "request_id": "3b08353e2e65e4a05c5d68a3061032db",
#   "latex_styled": "\\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right.",
#   "is_handwritten": true,
#   "confidence_rate": 0.47379128643166557,
#   "auto_rotate_degrees": 0,
#   "auto_rotate_confidence": 0.011462837151636762
# }

# this is what was returned
returned_ltx = "\\sum_{m}\\left(_{j m}^{2}+\\tan \\left(\\phi_{m}\\right)\\right."
md("$ %s $"%(returned_ltx))

$ \sum_{m}\left(_{j m}^{2}+\tan \left(\phi_{m}\right)\right. $

### Results

**PIL's Image**

<img src="figures/canvas_img_RB.png" alt="" title="" width="400" height="300" />

**Rendered LaTeX**

<img src="figures/ren_latex_RB.png" alt="" title="" width="400" height="300" />

**Original LaTeX**  
$\huge \sum_m y^2_m + \tan(\phi_m) $

We see that almost every character except the 'y' was converted correctly. This is in part due to bad hand-writing. Looking at how y is actually written, with a partial break at the stem, we can see how the OCR thought this was two seperate characters: one closely resembling a j and the second part of the curve resembing a parenthesis. Fortunately, a majority of the LaTeX was successfully converted, so a user could come back and fix this minor LaTeX issue.

## Test Case 1:

Take a closer look at how the drawing looks on the canvas vs the PIL's version of the drawing. The image on the right is saved and sent to the OCR. We see that PIL's image has more roughness to it, which may affect OCR accuracy. This is an issue that likely we will revisit in the coming development phases.

<p float="left">
  <img src="figures/img_6V3SBX4_cv.png" alt="" title="" width="400" height="300" />
  <img src="figures/img_6V3SBX4_pil.png" alt="" title="" width="400" height="300" /> 
</p>

## Test Case 2:

Hand-write and apply end-to-end solution to the following equation:

$$ \huge \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \nabla \cdot (\mathbf{v} c) + R $$

The file 'figures/canvas_img_YFXS3KF.png' is the saved version of my hand-drawn image. You can draw your own and use that file instead buy passing the correct argument to `ocr_request`.

In [1]:
# call the pilot_canvas() and set the return to a dictionary

filename = pilot_canvas()

# filename={"name":'figures/canvas_img_LKNB0E0.png'}

NameError: name 'pilot_canvas' is not defined

In [21]:
# send the API request
latex_return=ocr_request(filename)

{
    "auto_rotate_confidence": 0.02519008441753101,
    "auto_rotate_degrees": 0,
    "confidence": 0.3641139303316178,
    "confidence_rate": 0.3641139303316178,
    "is_handwritten": true,
    "is_printed": false,
    "latex_styled": "\\frac{\\partial_{c}}{\\partial t}=\\nabla \\cdot\\langle D \\nabla c\\rangle-\\nabla \\cdot(v c)+R",
    "request_id": "caae84fde22cc6bcb42fb597ce9ce9d3",
    "text": "\\( \\frac{\\partial_{c}}{\\partial t}=\\nabla \\cdot\\langle D \\nabla c\\rangle-\\nabla \\cdot(v c)+R \\)"
}
\frac{\partial_{c}}{\partial t}=\nabla \cdot\langle D \nabla c\rangle-\nabla \cdot(v c)+R



In [51]:
# print the returned request
md("$ %s $"%(latex_return))

$ \frac{\partial_{c}}{\partial t}=\nabla \cdot\langle D \nabla c\rangle-\nabla \cdot(v c)+R $

## Compare inputs and outputs to ideal

In [29]:
# compare to input image (PIL's image)
print(filename["name"])
# figures/canvas_img_YFXS3KF.png

figures/canvas_img_YFXS3KF.png


**PIL's Image**
<p float="left">
  <img src=figures/canvas_img_YFXS3KF.png width="300" />
</p>

**Rendered LaTeX Image**
<p float="left">
  <img src=figures/ren_latex_YFXS3KF.png width="350" />
</p>


**Original LaTeX**  
$ \LARGE \frac{\partial c}{\partial t} = \nabla \cdot (D \nabla c) - \nabla \cdot (\mathbf{v} c) + R $


### How did we do?

In test case 0, roughly 1/10 characters was incorrect, with 1 error not counted, since it's an extraneous character. In test case 2, 3/19 characters were incorrect, with the same mistake being made twice ('<' instead of '('). In test case 1, we did not translate the LaTeX but showed how the canvas drawing is being translated to a PIL image.

This leads us to think more about how we would evaluate a character-by-character measurement of accuracy. What if the translation introduces more characters than exist in the original? What if we have the correct character, but it's being placed incorrectly ($\partial c$ vs $\partial_c$)?

W